# Abstract

*A problem well stated is a problem half-solved.*

*This is your space to describe your intentions for the project, before writing a single line of code. What are you studying? What are you hoping to build? If you can't explain that clearly before you start digging into the data, you're going to have a hard time planning where to go with this.*

# Obtain the Data

*Describe your data sources here and explain why they are relevant to the problem you are trying to solve.*

*Your code should download the data and save it in data/raw. If you've got the data from an offline source, describe where it came from and what the files look like. Don't do anything to the raw data files just yet; that comes in the next step.*

*After completing this step, be sure to edit `references/data_dictionary` to include descriptions of where you obtained your data and what information it contains.*

In [3]:
## %%writefile ../src/data/make_dataset.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that download data from one or more sources
    and saves those datasets to the data/raw directory.
    """
    # download_dataset_1(url)
    # download_dataset_2(url)
    # save_dataset_1('data/raw', filename)
    # save_dataset_2('data/raw', filename)
    pass

In [25]:
from bs4 import BeautifulSoup
import urllib.request

url = "https://www.thespruceeats.com/a-to-z-cocktail-recipes-3962886"
resp = urllib.request.urlopen(url)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

# for link in soup.find_all('a', href=True):
#     print(link['href'])
print(soup)
soup.prettify()

<!DOCTYPE html>
<html class="comp structuredListTemplate html mntl-html" data-ab="99,99,99,99,43,99,99,4,1,11,99,86,20,91,99" data-lazy-threshold="120" data-lifestyle-food-resource-version="4.46.0" data-lifestyle-resource-version="4.46.0" data-mantle-resource-version="3.10.171" data-resource-version="4.46.0" id="structuredListTemplate_1-0" lang="en">
<!--
<globe-environment environment="prod" application="lifestyle-food" dataCenter="us-west-2"/>
-->
<head class="loc head">
<script type="text/javascript"></script> <link href="//adservice.google.com" rel="preconnect"/>
<link href="//securepubads.g.doubleclick.net" rel="preconnect"/>
<link href="//stats.g.doubleclick.net" rel="preconnect"/>
<link href="//js-sec.indexww.com" rel="preconnect"/>
<link href="//bcp.crwdcntrl.net" rel="preconnect"/>
<link href="//ad.crwdcntrl.net" rel="preconnect"/>
<link href="//c.amazon-adsystem.com" rel="preconnect"/>
<link href="//cdn.adsafeprotected.com" rel="preconnect"/>
<link href="//as-sec.casalemedia.

'<!DOCTYPE html>\n<html class="comp structuredListTemplate html mntl-html" data-ab="99,99,99,99,43,99,99,4,1,11,99,86,20,91,99" data-lazy-threshold="120" data-lifestyle-food-resource-version="4.46.0" data-lifestyle-resource-version="4.46.0" data-mantle-resource-version="3.10.171" data-resource-version="4.46.0" id="structuredListTemplate_1-0" lang="en">\n <!--\n<globe-environment environment="prod" application="lifestyle-food" dataCenter="us-west-2"/>\n-->\n <head class="loc head">\n  <script type="text/javascript">\n  </script>\n  <link href="//adservice.google.com" rel="preconnect"/>\n  <link href="//securepubads.g.doubleclick.net" rel="preconnect"/>\n  <link href="//stats.g.doubleclick.net" rel="preconnect"/>\n  <link href="//js-sec.indexww.com" rel="preconnect"/>\n  <link href="//bcp.crwdcntrl.net" rel="preconnect"/>\n  <link href="//ad.crwdcntrl.net" rel="preconnect"/>\n  <link href="//c.amazon-adsystem.com" rel="preconnect"/>\n  <link href="//cdn.adsafeprotected.com" rel="preconne

In [4]:
import urllib.request, json

In [8]:
with urllib.request.urlopen("https://www.thespruceeats.com/a-to-z-cocktail-recipes-3962886") as url:
    data = json.load(url)
    print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
newDictionary=json.loads(str(soup))
newDictionary

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import requests

In [10]:
url = "https://www.thespruceeats.com/a-to-z-cocktail-recipes-3962886"
r = requests.get(url)
# print(r.json())

In [23]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Scrub the Data

*Look through the raw data files and see what you will need to do to them in order to have a workable data set. If your source data is already well-formatted, you may want to ask yourself why it hasn't already been analyzed and what other people may have overlooked when they were working on it. Are there other data sources that might give you more insights on some of the data you have here?*

*The end goal of this step is to produce a [design matrix](https://en.wikipedia.org/wiki/Design_matrix), containing one column for every variable that you are modeling, including a column for the outputs, and one row for every observation in your data set. It needs to be in a format that won't cause any problems as you visualize and model your data.*

In [8]:
## %%writefile ../src/features/build_features.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/raw, cleans them,
    and converts the data into a design matrix that is ready for modeling.
    """
    # clean_dataset_1('data/raw', filename)
    # clean_dataset_2('data/raw', filename)
    # save_cleaned_data_1('data/interim', filename)
    # save_cleaned_data_2('data/interim', filename)
    # build_features()
    # save_features('data/processed')
    pass


*Before moving on to exploratory analysis, write down some notes about challenges encountered while working with this data that might be helpful for anyone else (including yourself) who may work through this later on.*

# Explore the Data

*Before you start exploring the data, write out your thought process about what you're looking for and what you expect to find. Take a minute to confirm that your plan actually makes sense.*

*Calculate summary statistics and plot some charts to give you an idea what types of useful relationships might be in your dataset. Use these insights to go back and download additional data or engineer new features if necessary. Not now though... remember we're still just trying to finish the MVP!*

In [ ]:
## %%writefile ../src/visualization/visualize.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed')
    # describe_features(data, 'reports/')
    # generate_charts(data, 'reports/figures/')
    pass


*What did you learn? What relationships do you think will be most helpful as you build your model?*

# Model the Data

*Describe the algorithm or algorithms that you plan to use to train with your data. How do these algorithms work? Why are they good choices for this data and problem space?*

In [ ]:
## %%writefile ../src/models/train_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed/')
    # train, test = train_test_split(data)
    # save_train_test(train, test, 'data/processed/')
    # model = build_model()
    # model.fit(train)
    # save_model(model, 'models/')
    pass


In [ ]:
## %%writefile ../src/models/predict_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # test_X, test_y = load_test_data('data/processed')
    # trained_model = load_model('models/')
    # predictions = trained_model.predict(test_X)
    # metrics = evaluate(test_y, predictions)
    # save_metrics('reports/')
    pass



_Write down any thoughts you may have about working with these algorithms on this data. What other ideas do you want to try out as you iterate on this pipeline?_

# Interpret the Model

_Write up the things you learned, and how well your model performed. Be sure address the model's strengths and weaknesses. What types of data does it handle well? What types of observations tend to give it a hard time? What future work would you or someone reading this might want to do, building on the lessons learned and tools developed in this project?_